In [ ]:
# installs
# pip install mlxtend

In [ ]:
# imports
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px

from mlxtend.frequent_patterns import apriori, association_rules

# my project in Google Cloud used
PROJECT = "questrom"

In [ ]:
# auth against your BU GCP account 
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

In [ ]:
# get the data from Big Query
# select all the records from 
# `questrom.datasets.groceries`
SQL = """
SELECT *
FROM `questrom.datasets.groceries`
"""

groceries = pd.io.gbq.read_gbq(SQL, PROJECT)

In [ ]:
# quick look
groceries.shape

In [ ]:
# first few rows
groceries.head(3)

#### Data Dictionary

- tid = transaction id
- item = the product

In [ ]:
## QUICK EXERCISE:
## take 3 minutes, how many unique transactions 
## and items?
## are there any duplicates?

print(groceries.tid.nunique())
print(groceries.item.nunique())
len(groceries) - len(groceries.drop_duplicates())

In [ ]:
### its worth noting that I tend to like this tidy format
### its easy to filter on a transaction and clearly see the detail
### it also works great when we have an element of time/sequence in our datasets

In [ ]:
## put the data into transaction format
## note that mlxtend wants a 1 row per transaction and 1-hot layout

# when we pivot, the intersections will be True
groceries['purchase'] = True

tx = groceries.pivot(index="tid", columns="item", values="purchase")

In [ ]:
# what do we have
tx.shape

In [ ]:
tx.head(3)

In [ ]:
# we need to fill in the missings, which is easy with pandas
tx.fillna(value=False, inplace=True)
tx.head(3)

In [ ]:
# plot the items per transaction
item_count = tx.sum(axis=1)

print(item_count.shape)

item_count.value_counts(ascending=False).plot(kind="bar")

In [ ]:
## why does this help us understand the dataset?

In [ ]:
# understanding how often a product exists will help us with pruning
item_freq = tx.sum(axis=0)

print(item_freq.shape)

# normalize against all transactions
item_freq = item_freq / len(tx)

# plot
sns.histplot(item_freq)

In [ ]:
# summarize the distro
item_freq.describe()

In [ ]:
## what does this tell us about suppport settings?

In [ ]:
## QUICK EXERCISE:  What are the top 5 products?
item_freq.sort_values(ascending=False).head()


In [ ]:
# apply the apriori algorithm to the dataset
itemsets = apriori(tx, min_support=.003, use_colnames=True)

In [ ]:
# what do we have?
type(itemsets)

In [ ]:
itemsets.head(3)

In [ ]:
## lets fit our first assoc rules model!
## http://rasbt.github.io/mlxtend/api_subpackages/mlxtend.frequent_patterns/

## leaving confidence low to demonstrate rule evaluation
# itemsets.drop(columns="length", inplace=True)
rules = association_rules(itemsets, metric='confidence', min_threshold=.2)

In [ ]:
# what do we have
type(rules)

In [ ]:
# first few
rules.head(3)

In [ ]:
# mlxtend uses frozensets, whichm are sets but immutable

# lets look for the RHS that hsa bottled water
rules.loc[rules.consequents == {'bottled water'}, :].head()

In [ ]:
## Exercise:  5 minutes, think about the following questions
##   how many rules were created
##   what is the range of lift values
##   what is the average rule size (how many items)
##      HINT:  lambdas 
##   plot (barplot) of rule sizes

print(len(rules))
print(rules.lift.max() - rules.lift.min())
rules['length'] = rules.antecedents.apply(lambda x: len(x)) + rules.consequents.apply(lambda x: len(x))
print(rules.length.mean())

In [ ]:
## barplot of rule sizes
rule_length = rules.length.value_counts()
sns.barplot(rule_length.index, rule_length.values)

In [ ]:
# lets look at the first few rows
rules.head()

In [ ]:
## EXERCISE:
## calculate the support for the rule below
## HINT:  you can use the prior datasets
## The Rule: Instant food products -> hamburger meat

filter = np.where((tx['Instant food products']==True) & (tx['hamburger meat']==True))
len(filter[0]) / len(tx)



In [ ]:
# of course, because we have a pandas dataframe, its very simple to inspect

In [ ]:
## Exercise - Take 5 minutes
## 1.  sort the rules ascending by lift - print out the first 5
## 2.  find the top 10 rules (sorted descending by support) where chewing gum
##     is in the RHS

## 1
rules.sort_values("lift", ascending=False).head(5)


## 2 
rules.loc[rules.consequents=={'soda'}, :].sort_values("support", ascending=False).head(10)

In [ ]:
## QUICK QUESTION:  From an analyst point of view, why might we want to 
##                  filter the rules from the start?  Example applications of
##                  where we might do this?



In [ ]:
## Visualize the rules 
## support and lift
## as DC noted, this can help you with filtering if you start relatively unpruned
px.scatter(rules, x="antecedent support", y="consequent support", size="lift")

In [ ]:
## we could also look at support/confidence/lift
plt.figure(figsize=(10,4))
sns.scatterplot(data=rules, x="support", y="confidence", hue="lift", alpha=.7)